In [16]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [17]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [18]:
#create new table for regions

regions = pd.read_csv('Untitled.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS regions')
c.execute('CREATE TABLE regions (state str, census_bureau_region str, census_bureau_division str, standard_federal_region str, circuit_court_appeals str, bureau_economic_analysis str)')
conn.commit()

regions.to_sql('regions',conn, if_exists='replace')

/opt/anaconda3/envs/amanda/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [19]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
states
us
election
counties
regions


In [20]:
df = pd.read_sql('''select * from regions''',conn)
df.columns

Index(['State', 'census_bureau_region', 'census_bureau_division',
       'standard_federal_region', 'circiut_court_of_appeals',
       'bureau_economic_analysys', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

In [21]:
df = pd.read_sql('''select * from election''',conn)
df

,state,county,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,demsen16,...,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
0,Alabama,Autauga,1001,18172,5936,865,17379,6363,190,6331.0,...,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,10.002112,74.065601,42.002162,2.0
1,Alabama,Baldwin,1003,72883,18458,3874,66016,18424,898,19145.0,...,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,7.842227,68.405607,42.279099,3.0
2,Alabama,Barbour,1005,5454,4871,144,5550,5912,47,4777.0,...,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,19.579752,81.364746,67.789635,6.0
3,Alabama,Bibb,1007,6738,1874,207,6132,2202,86,2082.0,...,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,15.020490,87.471774,68.352607,1.0
4,Alabama,Blount,1009,22859,2156,573,20757,2970,279,2980.0,...,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,16.643368,86.163610,89.951502,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,Wyoming,Sweetwater,56037,12154,3231,1745,11428,4774,693,NaN,...,44.153352,9.417120,68233.0,5.072255,9.314606,78.628507,6.238463,76.606813,10.916313,5.0
3110,Wyoming,Teton,56039,3921,7314,1392,4858,6213,393,NaN,...,35.569995,11.837510,75594.0,2.123447,4.633570,46.211584,1.526877,41.769504,46.430920,7.0
3111,Wyoming,Uinta,56041,6154,1202,1114,6615,1628,296,NaN,...,43.205858,10.678218,53323.0,6.390755,10.361224,81.793082,8.806312,81.080852,43.095937,7.0
3112,Wyoming,Washakie,56043,2911,532,371,3014,794,136,NaN,...,34.774279,19.650341,46212.0,7.441860,12.577108,78.923920,10.299738,75.980688,35.954529,7.0


In [22]:
df.columns

Index(['state', 'county', 'fips', 'trump16', 'clinton16', 'otherpres16',
       'romney12', 'obama12', 'otherpres12', 'demsen16', 'repsen16',
       'othersen16', 'demhouse16', 'rephouse16', 'otherhouse16', 'demgov16',
       'repgov16', 'othergov16', 'repgov14', 'demgov14', 'othergov14',
       'total_population', 'cvap', 'white_pct', 'black_pct', 'hispanic_pct',
       'nonwhite_pct', 'foreignborn_pct', 'female_pct', 'age29andunder_pct',
       'age65andolder_pct', 'median_hh_inc', 'clf_unemploy_pct', 'lesshs_pct',
       'lesscollege_pct', 'lesshs_whites_pct', 'lesscollege_whites_pct',
       'rural_pct', 'ruralurban_cc'],
      dtype='object')

In [23]:
df = pd.read_sql('''SELECT c.state, c.county, c.fips,
                    cases - LAG (cases,1) OVER (PARTITION BY c.state, c.county ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (PARTITION BY c.county ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths', 
                    bureau_economic_analysys as 'BEA_Region',
                    e.total_population
                    FROM counties as c
                    JOIN regions r on c.state = r.state
                    JOIN election e on e.fips = c.fips
                    ORDER BY c.state, c.county, date''',conn)
df

,state,county,fips,new_cases,cumulative_cases,new_deaths,cumulative_deaths,BEA_Region,total_population
0,Alabama,Autauga,1001.0,NaN,1,NaN,0,Southeast,55049.0
1,Alabama,Autauga,1001.0,3.0,4,0.0,0,Southeast,55049.0
2,Alabama,Autauga,1001.0,2.0,6,0.0,0,Southeast,55049.0
3,Alabama,Autauga,1001.0,0.0,6,0.0,0,Southeast,55049.0
4,Alabama,Autauga,1001.0,0.0,6,0.0,0,Southeast,55049.0
...,...,...,...,...,...,...,...,...,...
663270,Wyoming,Weston,56045.0,9.0,148,0.0,0,Rocky Mountain,7175.0
663271,Wyoming,Weston,56045.0,7.0,155,0.0,0,Rocky Mountain,7175.0
663272,Wyoming,Weston,56045.0,4.0,159,0.0,0,Rocky Mountain,7175.0
663273,Wyoming,Weston,56045.0,4.0,163,0.0,0,Rocky Mountain,7175.0


In [24]:
df = pd.read_sql('''WITH cte as (SELECT c.date, c.state, c.county,
                                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                                    cases as 'cumulative_cases'
                    FROM     counties c)
                    
SELECT date,
       state, 
       county, 
       new_cases, 
       cumulative_cases,
       AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new'
FROM cte
ORDER BY state, county, date
LIMIT 25''',conn)
df

,date,state,county,new_cases,cumulative_cases,7-day_avg_new
0,2020-03-24,Alabama,Autauga,NaN,1,NaN
1,2020-03-25,Alabama,Autauga,3.0,4,3.000000
2,2020-03-26,Alabama,Autauga,2.0,6,2.500000
3,2020-03-27,Alabama,Autauga,0.0,6,1.666667
4,2020-03-28,Alabama,Autauga,0.0,6,1.250000
5,2020-03-29,Alabama,Autauga,0.0,6,1.000000
6,2020-03-30,Alabama,Autauga,1.0,7,1.000000
7,2020-03-31,Alabama,Autauga,0.0,7,0.857143
8,2020-04-01,Alabama,Autauga,3.0,10,0.857143
9,2020-04-02,Alabama,Autauga,0.0,10,0.571429
